# Langchain Quickstart
https://venelinvalkov.medium.com/langchain-quickstart-build-powerful-chatgpt-applications-4d3bdbc742b2

## Setup

In [1]:
import os
import textwrap
 
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader

In [2]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

## Model


In [3]:
model = OpenAI(temperature=0)

In [4]:
print(
    model(
        "You're Dwight K. Schrute from the Office. Suggest 5 places to visit in Scranton that are connected to the TV show."
    )
)



1. The Dunder Mifflin Paper Company - Visit the office building where the show was filmed and take a tour of the set.

2. Poor Richard's Pub - Enjoy a drink at the bar where the cast often hung out.

3. Steamtown National Historic Site - Take a ride on the historic train that was featured in the show.

4. The Scranton Cultural Center - Attend a show at the theater where the cast performed a play in the episode "The Duel".

5. The Mall at Steamtown - Shop at the mall where the cast went on a shopping spree in the episode "The Coup".


### Data loader

In [5]:
loader = WebBaseLoader(
    "https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm"
)

In [6]:
documents = loader.load()
len(documents)

1

In [7]:
document = documents[0]
document.__dict__.keys()

dict_keys(['page_content', 'metadata'])

In [8]:
document.page_content[:100]

"\n\n\n\n\nTwitter's Recommendation Algorithm\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEngineering\n\n\n\n\n\nBac"

In [9]:
document.metadata

{'source': 'https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm',
 'title': "Twitter's Recommendation Algorithm",
 'description': 'Twitter aims to deliver you the best of what’s happening in the world right now. This blog is an introduction to how the algorithm selects Tweets for your timeline.',
 'language': 'en_us'}

In [10]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [11]:
query = """
You're Jarvis from Iron Man

Explain the Twitter recommendation algorithm in 5 sentences.
"""

print_response(index.query(query))

 The Twitter recommendation algorithm is composed of many interconnected services and jobs that aim
to distill the roughly 500 million Tweets posted daily down to a handful of top Tweets that show up
on your device’s For You timeline. The foundation of the algorithm is a set of core models and
features that extract latent information from Tweet, user, and engagement data. The recommendation
pipeline is made up of three main stages: fetching the best Tweets from different recommendation
sources, ranking each Tweet using a machine learning model, and applying heuristics and filters. The
service that is responsible for constructing and serving the For You timeline is called Home Mixer,
which is built on Product Mixer, a custom Scala framework. Home Mixer acts as the software backbone
that connects different candidate sources, scoring functions, heuristics, and filters.


In [12]:
template = """You're a personal assistant, Jane with full integration to iCloud accounts and devices.
 
{context}
 
Help get things done by executing commands like:

- Open the calculator app
- Open the calendar app
- Send a text to John and tell him I'll be late
- Read the last message from John
 
Question: {question}
Answer:"""

In [13]:
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
)

print(prompt.format(context="It's Tuesday morning and I'm about to head out to work", question="What's my next appointment?"))

You're a personal assistant, Jane with full integration to iCloud accounts and devices.
 
It's Tuesday morning and I'm about to head out to work
 
Help get things done by executing commands like:

- Open the calculator app
- Open the calendar app
- Send a text to John and tell him I'll be late
- Read the last message from John
 
Question: What's my next appointment?
Answer:


In [14]:
embedding = OpenAIEmbeddings()
db = Chroma.from_documents(documents, embedding)

In [15]:
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.6),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [16]:
query = "Explain the Twitter recommendation algorithm in 5 sentences."
response = chain.run(query)

In [17]:
print_response(response)

Twitter's recommendation algorithm distills the 500 million daily tweets down to a handful of top
tweets for a user's timeline. The algorithm is composed of interconnected services and jobs that
extract latent information from tweet, user, and engagement data. The recommendation pipeline has
three main stages: candidate sourcing, ranking, and applying heuristics and filters. The Home Mixer
service constructs and serves the For You timeline, and the algorithm uses a neural network to
continuously train on tweet interactions and optimize for positive engagement. The algorithm also
applies various product features, such as visibility filtering and social proof, to create a
balanced and diverse feed.
